In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, pi, atan2, asin, sqrt
import matplotlib.pyplot as plt
from scipy import integrate
from model_funcs import *

In [5]:
def nasa_pres(P, P0, T0, R, B, g):
        imu_temp = T0*(P/P0)**(R*B/g)
        imu_alt = (T0 - imu_temp)/B
        return imu_alt
    

def calc_moving_avg(axg21t, n, tdata21, dynamic_window=False, dynamic_n_timing=140, dynamic_n=80):        
    if dynamic_window:
        axg21s_1 = pd.Series(axg21t[0:dynamic_n_timing]).rolling(window=n).mean().iloc[n-1:].values
        axg21s_2 = pd.Series(axg21t).rolling(window=dynamic_n).mean().iloc[n-1:].values[dynamic_n_timing:]
        new_axg21s = list(axg21s_1) + list(axg21s_2)
    else:
        axg21s = pd.Series(axg21t).rolling(window=n).mean().iloc[n-1:].values
        new_axg21s = list(axg21s)
    while len(new_axg21s) < len(tdata21):
        new_axg21s = [0] + list(new_axg21s) + [0]
    return new_axg21s

def find_peak_and_wait(a_vec, t, thresh=40, post_drogue_delay=1, signal_length=3, time=1, t_sim_drogue=9.85, t_sim_landing=90):
    '''
    USE Z FOR DETECTION BUT THE SIGNAL WE WANT IS IN X AND Y
    '''
    axn, ayn, azn = np.array(a_vec[0], float), np.array(a_vec[1], float), np.array(a_vec[2], float)
        
    if time==0:
        above_threshold = abs(azn) > thresh
        takeoff_idx = list(above_threshold).index(True)

        # Get past the full takeoff spike
        post_takeoff_idx = takeoff_idx
        if azn[post_takeoff_idx] > 0:
            flip = -1
        else:
            flip = 1
        takeoff_offset = 0
        while (azn[post_takeoff_idx]*flip < 0):
            post_takeoff_idx += 1
        
        # NOW FIND DROGUE
        above_threshold = abs(azn[post_takeoff_idx:]) > thresh
        takeoff_idx = list(above_threshold).index(True)
        
        # Fix this for fullscale.  Subscale won't work cause there's not actual drogue spike
        ########################################################################
        # Now get past the transcience
        while t[past_takeoff_idx] < t[takeoff_idx+takeoff_offset]:
            past_takeoff_idx += 1

        end_drogue_idx = drogue_idx
        while t[end_drogue_idx] < (t[drogue_idx] + signal_length):
            end_drogue_idx += 1
            
        ax_1 = np.average(axn[drogue_idx:end_drogue_idx])
        ay_1 = np.average(ayn[drogue_idx:end_drogue_idx])
        a_1 = [ax_1, ay_1]
            
        landing_idx = end_drogue_idx
        while t[landing_idx] < t_sim_landing:
            landing_idx += 1

        return takeoff_idx, drogue_idx, end_drogue_idx, landing_idx, a_1
        ########################################################################
    elif time==1:
        # Detect takeoff
        takeoff_idx = np.argmax(abs(azn)>thresh)
        drogue_idx = takeoff_idx
        
        # Wait until sim_drogue
        t_takeoff = t[takeoff_idx]
        t_drogue = t_takeoff
        while t_drogue < (t_takeoff + t_sim_drogue):
            drogue_idx += 1
            t_drogue = t[drogue_idx]
            
        # Now wait until takeoff_delay passes
        t_signal = t_drogue
        end_drogue_idx = drogue_idx
        while t_signal < (t_drogue + post_drogue_delay):
            end_drogue_idx += 1
            t_signal = t[end_drogue_idx]
            
        # Get the resting acceleration
        ax_1 = np.average(axn[drogue_idx:end_drogue_idx])
        ay_1 = np.average(ayn[drogue_idx:end_drogue_idx])
        a_1 = [ax_1, ay_1]
        
        # Find the landing based on time
        landing_idx = end_drogue_idx
        while t[landing_idx] < t_sim_landing + t_takeoff:
            landing_idx += 1
        
        return takeoff_idx, drogue_idx, end_drogue_idx, landing_idx, a_1
    else:
        print("BAD INPUT - FIX TIME PARAMETER")
        return -1, -1, -1, -1, -1

In [12]:
def calc_displacement2(datafile, zero_out=False, my_thresh=50, my_post_drogue_delay=1, my_signal_length=3, use_time=True, my_t_sim_drogue=9.85, my_t_sim_landing=50):
    '''
    1. Input the file, read the data in
    2. Use find_peak_and_wait to find the drogue peak and isolate the signal
    - Revamp this to use altitude instead of time?
    3. Plot altitude vs time
    4. 
    '''

    # Parameters
    dt = 0.001
    B = 6.5*10**-3   # temperature lapse rate in troposphere in K/m
    R = 287   # ideal gas constant in J/(kg.K)
    g = 9.80665  # gravity at sea level in m/s2
    T0 = 288.15   # standard air temperature in K
    P0 = 101.325   # standard air pressure in kPa
    pi = 3.1415
    ft = 3.2884  # ft/m
    ms2mph = 0.6818182*ft
    gs2mph = ms2mph * 9.8

    # Read in the dataframe
    fields = ['Timestamp', 'Pres',
    'Roll', 'Pitch', 'Yaw',
    'LinearAccelNed X', 'LinearAccelNed Y', 'LinearAccelNed Z']
    #datafile = '../Data/Fullscale2/' + datafile + '.csv'
    df = pd.read_csv(datafile, skipinitialspace=True, usecols=fields)

    # Read Data Fields
    imu_t = df['Timestamp'].values
    dt = imu_t[1]
    imu_N = len(imu_t)
    imu_ax = df['LinearAccelNed X'].values
    imu_ay = df['LinearAccelNed Y'].values
    imu_az = df['LinearAccelNed Z'].values * -1
    imu_pres = df['Pres']

    ################## INIT VECTORS  ##################
    imu_vx, imu_vy, imu_vz, imu_x, imu_y, imu_z = (np.zeros((imu_N)), np.zeros((imu_N)), 
                                                   np.zeros((imu_N)), np.zeros((imu_N)), 
                                                   np.zeros((imu_N)), np.zeros((imu_N)))
    imu_vx_flight, imu_vy_flight, imu_vz_flight, imu_x_flight, imu_y_flight, imu_z_flight = (np.zeros((imu_N)), np.zeros((imu_N)), 
                                                   np.zeros((imu_N)), np.zeros((imu_N)), 
                                                   np.zeros((imu_N)), np.zeros((imu_N)))

    # Find drogue peak and calc wind velocity
    a_vec = [imu_ax, imu_ay, imu_az]
    takeoff_time, imu_start_time, imu_end_time, landing_idx, a_1 = find_peak_and_wait(a_vec, imu_t, thresh=my_thresh, post_drogue_delay=my_post_drogue_delay, signal_length=my_signal_length, time=use_time, t_sim_drogue=my_t_sim_drogue, t_sim_landing=my_t_sim_landing)
    # ^ RETURNED VALUES ARE ALL INDICES NOT ACTUAL TIMES
    ax_1, ay_1 = a_1[0], a_1[1]

    ################## Find alt  ##################
    vec_NASA_pres = np.vectorize(nasa_pres)
    imu_alt = vec_NASA_pres(imu_pres, P0, T0, R, B, g)
    imu_alt = imu_alt - imu_alt[0]
    imu_alt = [val if val > 0 else 0 for val in imu_alt]

    ## TRUNCATE ALL THE VECTORS
    # This is the truncated signal we're interested in
    imu_ax_flight = imu_ax[takeoff_time:landing_idx]
    imu_ay_flight = imu_ay[takeoff_time:landing_idx]
    imu_az_flight = imu_az[takeoff_time:landing_idx]
    imu_alt = imu_alt[takeoff_time:landing_idx]
    imu_t_flight = imu_t[takeoff_time:landing_idx]
    imu_t_flight = imu_t_flight - imu_t_flight[0]

    # Find the max altitude
    z_0 = max(imu_alt)
    apogee_idx = list(imu_alt).index(z_0)
    # find_peak_and_wait returns the "signal" start and finish times
    # Alternatively, we could also add 1 second to the apogee_idx and find the corresponding index
    temp = imu_t_flight - imu_t_flight[apogee_idx]  - my_post_drogue_delay
    imu_start_time = list(temp).index(min(abs(temp)))

    temp = imu_t_flight - imu_t_flight[apogee_idx] - my_post_drogue_delay - my_signal_length
    imu_end_time = list(temp).index(min(abs(temp)))

    dt = imu_t_flight[1]

    imu_ax_signal = imu_ax_flight[imu_start_time:imu_end_time]
    imu_ay_signal = imu_ay_flight[imu_start_time:imu_end_time]
    w0x = integrate.trapz(imu_ax_signal, dx=dt)
    w0y = integrate.trapz(imu_ay_signal, dx=dt)
    print(f"WIND SPEEDS, X->{w0x} m/s and Y->{w0y} m/s")

    # Find the displacement after imu_end_time
    ################## Find velocity and position  ##################
    for i in range(len(imu_t_flight)-1):
        imu_vz_flight[i+1] = imu_vz_flight[i] + imu_az_flight[i]*(imu_t_flight[i+1] - imu_t_flight[i])
        imu_z_flight[i+1] = imu_z_flight[i] + imu_vz_flight[i]*(imu_t_flight[i+1] - imu_t_flight[i])

        imu_vx_flight[i+1] = imu_vx_flight[i] + imu_ax_flight[i]*(imu_t_flight[i+1] - imu_t_flight[i])
        imu_x_flight[i+1] = imu_x_flight[i] + imu_vx_flight[i]*(imu_t_flight[i+1] - imu_t_flight[i])

        imu_vy_flight[i+1] = imu_vy_flight[i] + imu_ay_flight[i]*(imu_t_flight[i+1] - imu_t_flight[i])
        imu_y_flight[i+1] = imu_y_flight[i] + imu_vy_flight[i]*(imu_t_flight[i+1] - imu_t_flight[i])

    drogue_opening_displacement_x = imu_x_flight[imu_end_time] - imu_x_flight[imu_start_time]
    drogue_opening_displacement_y = imu_y_flight[imu_end_time] - imu_y_flight[imu_start_time]

    final_x_displacements, final_y_displacements = [0]*3, [0]*3
    for idx, uncertainty in enumerate([-1, 0, 1]):
        #For end_time to landing
        total_x_displacement = 0
        total_y_displacement = 0
        for i in range(imu_end_time, landing_idx-takeoff_time):
            vx = (w0x+uncertainty)*((imu_alt[i]/z_0)**(1/7))
            vy = (w0y+uncertainty)*((imu_alt[i]/z_0)**(1/7))
            total_y_displacement += vy*(imu_t_flight[i] - imu_t_flight[i-1])
            total_x_displacement += vx*(imu_t_flight[i] - imu_t_flight[i-1])

        # Oz Ascent Model
        final_x_displacements[idx] = (imu_x_flight[imu_start_time] - imu_x_flight[takeoff_time]) + drogue_opening_displacement_x + total_x_displacement
        final_y_displacements[idx] = (imu_y_flight[imu_start_time] - imu_y_flight[takeoff_time]) + drogue_opening_displacement_y + total_y_displacement

        # Marissa Ascent Model
        #...

        # Take max and min of ALL 6 --> Then average for final result

        print(f"TOTAL X AND Y DISPLACEMENTS for correction of {uncertainty} m/s to windspeed: X->{final_x_displacements[idx]:2f} m, Y->{final_y_displacements[idx]:2f} m")

    #return [x1[landing_idx], x2[landing_idx]], [y1[landing_idx], y2[landing_idx]], [x1, x2, t1, t2, y1, y2, ty1, ty2], [takeoff_time, imu_start_time, imu_end_time, landing_idx]
    return np.mean(final_x_displacements), np.mean(final_y_displacements)

In [13]:
LSM_datafile = "LSM_LOG_20220402-132105"
SIFT1_datafile = "SIFT1_LOG_20220402_124920"
SIFT2_datafile = "SIFT2_LOG_20220402_124347"
Full_LSM_datafile = "FULL_LSM"
Full_SIFT1_datafile = "Full_SIFT1"
Full_SIFT2_datafile = "Full_SIFT2"

In [14]:
datafile = '../../' + Full_SIFT2_datafile + '.csv'

x_S2F2, y_S2F2 = calc_displacement2(datafile, use_time=True, my_t_sim_drogue=9.85, my_t_sim_landing=50)
print(x_S2F2, y_S2F2, sqrt((x_S2F2**2) + (y_S2F2**2)))

WIND SPEEDS, X->4.6743435180311765 m/s and Y->-1.0590806853070638 m/s
TOTAL X AND Y DISPLACEMENTS for correction of -1 m/s to windspeed: X->125.969352 m, Y->-91.985122 m
TOTAL X AND Y DISPLACEMENTS for correction of 0 m/s to windspeed: X->154.852405 m, Y->-63.102069 m
TOTAL X AND Y DISPLACEMENTS for correction of 1 m/s to windspeed: X->183.735457 m, Y->-34.219017 m
154.8524048819776 -63.10206919532427 167.21584384998755


In [15]:
datafile = '../../' + Full_SIFT1_datafile + '.csv'

x_S1F1, y_S1F1 = calc_displacement2(datafile, use_time=True, my_t_sim_drogue=9.85, my_t_sim_landing=50)
print(x_S1F1, y_S1F1, sqrt((x_S1F1**2) + (y_S1F1**2)))

IndexError: index 90198 is out of bounds for axis 0 with size 90198

In [ ]:
datafile = '../Data/Fullscale1.csv'

x_F1, y_F1 = calc_displacement2(datafile, use_time=True, my_t_sim_drogue=9.85, my_t_sim_landing=50)
print(x_F1, y_F1, sqrt((x_F1**2) + (y_F1**2)))

In [ ]:
Full_SIFT2_datafile = "Full_SIFT2"
datafile = '../Data/Subscale2/VN/LOG_20220130_014355.csv'

x_S2, y_S2 = calc_displacement2(datafile, use_time=True, my_t_sim_drogue=9.85, my_t_sim_landing=50)
print(x_S2, y_S2, sqrt((x_S2**2) + (y_S2**2)))